In [20]:
# Generate triples default parameters
# kgtk_path takes in the directory of containing KGTK edge files
kgtk_path = '/Users/rijulvohra/Documents/work/Novartis-ISI/kgtk_development/data/Q28885102'
output_filename = 'pharma_product_concat.tsv.gz'
triple_filename = 'pharma_triple.ttl'
triple_generation_log = 'pharma_log.txt'
properties_file_path = './properties.tsv'

In [21]:
import os
import re
import subprocess
import gzip
dirname = os.path.abspath('')
kgtk_path = os.path.join(dirname,kgtk_path)
print(kgtk_path)

/Users/rijulvohra/Documents/work/Novartis-ISI/kgtk_development/data/Q28885102


In [3]:
def get_uncompressed_size(file):
    pipe_in = os.popen('gzip -l %s' % file)
    list_1 = pipe_in.readlines()
    list_2 = list_1[1].split()
    c , u , r , n = list_2
    return int(u)

In [10]:
%%time
def find_files(path):
    kgtk_files = []
    for file_name in os.listdir(path):
        if file_name.startswith('Q') and file_name.endswith('.gz'):
            if re.search('.statistics.tsv.gz',file_name) or re.search('.P279star.tsv.gz',file_name):
                continue
            if get_uncompressed_size(os.path.join(kgtk_path,file_name)):
                kgtk_files.append(os.path.join(kgtk_path,file_name))
    return kgtk_files
kgtk_files = find_files(kgtk_path)

CPU times: user 10.7 ms, sys: 29.2 ms, total: 39.9 ms
Wall time: 140 ms


In [11]:
%%time
def generate_concat_files(kgtk_files, path, output_filename):
    concat_input = ' '.join(kgtk_files)
    output_concat = os.path.join(kgtk_path,output_filename)
    !kgtk cat -o $output_concat -i $concat_input
generate_concat_files(kgtk_files, kgtk_path, output_filename)

CPU times: user 19.6 ms, sys: 11.7 ms, total: 31.4 ms
Wall time: 1.39 s


In [13]:
##generate_wikidata_triples

gen_triple_input = os.path.join(kgtk_path,output_filename)
triple_output_save_path = os.path.join(kgtk_path,triple_filename)
log_save_path = os.path.join(kgtk_path,triple_generation_log)
!time cat $gen_triple_input | kgtk generate_wikidata_triples -ap aliases,alias -lp label -dp description \
                                                             -pf $properties_file_path \
                                                             -n 100000 \
                                                             --debug \
                                                             -gt yes -gz yes -w yes \
                                                             -log $log_save_path > $triple_output_save_path

cat   0.00s user 0.00s system 0% cpu 9.485 total
kgtk generate_wikidata_triples -ap aliases,alias -lp label -dp description -p  21.75s user 0.59s system 101% cpu 21.925 total


In [14]:
%%time
!gzip -f $triple_output_save_path

CPU times: user 4.37 ms, sys: 8.14 ms, total: 12.5 ms
Wall time: 233 ms
